In [1]:
#installing packages
!pip install datasets==1.13.3
!pip install transformers==4.11.3
!pip install torchaudio
!pip install librosa
!pip install jiwer

In [2]:
#importing datsets
from datasets import load_dataset


In [3]:
#loading librispeech clean dataset
dataset = load_dataset('librispeech_asr', 'clean')

Downloading:   0%|          | 0.00/2.28k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

  0%|          | 0/4 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/338M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset librispeech_asr downloaded and prepared to /root/.cache/huggingface/datasets/librispeech_asr/clean/2.1.0/f58a17d82b157feddbc407342a657fee7389b87c33d8331312edf7a604392896. Subsequent calls will reuse this data.


  0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
#printing the shape of the dataset
dataset.shape

{'test': (2620, 6),
 'train.100': (28539, 6),
 'train.360': (104014, 6),
 'validation': (2703, 6)}

In [5]:
#importing other packages from the dataset
from datasets import load_metric, Audio


In [6]:
#loading test dataset
dataset_test = load_dataset('librispeech_asr', 'clean',split = 'test')

Reusing dataset librispeech_asr (/root/.cache/huggingface/datasets/librispeech_asr/clean/2.1.0/f58a17d82b157feddbc407342a657fee7389b87c33d8331312edf7a604392896)


In [7]:
#shape of the test dataset
dataset_test.shape

(2620, 6)

In [8]:
#splitting the dataset on basis of train and validation
dataset_train = load_dataset('librispeech_asr', 'clean',split = 'train.100+validation')

Reusing dataset librispeech_asr (/root/.cache/huggingface/datasets/librispeech_asr/clean/2.1.0/f58a17d82b157feddbc407342a657fee7389b87c33d8331312edf7a604392896)


In [9]:
#details of the dataset
dataset_train,dataset_test

(Dataset({
     features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
     num_rows: 31242
 }), Dataset({
     features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
     num_rows: 2620
 }))

In [10]:
#shape of the train dataset
dataset_train.shape

(31242, 6)

In [13]:
#removing unnecessary columns from the datasets
dataset_train = dataset_train.remove_columns(["file", "speaker_id", "chapter_id", "id"])
dataset_test = dataset_test.remove_columns(["file", "speaker_id", "chapter_id", "id"])

In [14]:
#loading the dataset_train features after removing some features from the original dictionary
dataset_train

Dataset({
    features: ['audio', 'text'],
    num_rows: 31242
})

In [15]:
#loading the packages to view the random examples of the dataset
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [16]:
# displaying the 10 random elements of the dataset
show_random_elements(dataset_train.remove_columns(["audio"]), num_examples=10)

,text
0,AND THEY JOURNEYED ON WITH THEIR LITTLE WHIP THEY HAD AS MUCH MONEY AS THEY WANTED WORE GRAND CLOTHES AND MADE THEIR WAY INTO THE WORLD WHEREVER THEY WENT THEY LIVED IN MERRYMAKING AND SPLENDOUR DROVE ABOUT WITH HORSES AND CARRIAGES ATE AND DRANK BUT DID NOTHING WRONG
1,SAID THE OLD WOMAN I AM GOING OVER THE HILL TOP MYSELF AND I WILL SEND THE SHEEP BACK TO YOU SO SHE GOT UPON HER FEET AND BEGAN CLIMBING THE HILL AND THE GIRL HEARD HER SAYING AS SHE WALKED AWAY LITTLE BO PEEP HAS LOST HER SHEEP
2,THINKING RIGHTLY THAT THE TOWN WHICH HAD GIVEN THEM BIRTH RATHER THAN THE TOWN OF THEIR ADOPTION SHOULD FEED THEM AND KEEP A ROOF OVER THEIR HEADS UNTIL THE CHILDREN WERE OF AN AGE FOR SELF SUPPORT THERE WAS LITTLE TO EAT IN THE HOUSEHOLD
3,THAT THOUGH THERE ARE MANY ANIMALS WHICH MANIFEST MORE INDUSTRY THAN WE IN CERTAIN OF THEIR ACTIONS THE SAME ANIMALS ARE YET OBSERVED TO SHOW NONE AT ALL IN MANY OTHERS SO THAT THE CIRCUMSTANCE THAT THEY DO BETTER
4,ARE DERIVED FROM A RADICAL SIGNIFYING WHITE
5,YOU MUST MEAN YOUR UNCLE'S REPLIED MY GREAT GRANDMOTHER MY GREAT GRANDFATHER WAS CONVINCED HE UNTWISTED THE PAPER AND SIGNED THE ARTICLE
6,AND YOU'LL HAVE THE SUN IN HIS EYES BEHOLD US ONCE MORE FACE TO FACE LIKE DAVID AND THE PHILISTINE LOOK AT US AS LONG AS YOU MAY FOR THIS IS ALL YOU SHALL SEE OF THE COMBAT
7,THESE EXTERNAL TRAPPINGS OF THE SERVICE SOUND DEPRESSING ENOUGH BUT IF THE MESSAGE RECEIVED WITHIN THESE CHILLY WALLS IS CHEERING MAYBE WE CAN FORGET OR IGNORE THE PHYSICAL DISCOMFORTS BUT IS THE MESSAGE CHEERING HELL DAMNATION ETERNAL TORTURES
8,KNOWING THAT SHE HAD MARRIED WALTER GORE AND ALTHOUGH THERE WAS A FLAW IN THE CIRCUMSTANCES UPON WHICH SHE BUILT HER CLAIM YET SHE TRUSTED TO HER OWN CLEVERNESS TO CONCEAL THIS FROM THE TOO CLEVER LAWYER BUT APART FROM THIS
9,FRANK WAS ALWAYS ACCOMMODATING ESPECIALLY WHEN ANYTHING CONNECTED WITH HIS KNOWLEDGE OF NATURE WAS CONCERNED HE LOVED TO WATCH THE SMALL WOODS FOLK WHEN THEY DID NOT SUSPECT HIS PRESENCE AND LEARN MORE AND MORE OF THEIR INTERESTING HABITS


In [17]:
# preprocessing the dataset using the regular expressions 
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

In [18]:
# applying the function to remove the special characters from the dataset
dataset_train = dataset_train.map(remove_special_characters)
dataset_test = dataset_test.map(remove_special_characters)

  0%|          | 0/31242 [00:00<?, ?ex/s]

  0%|          | 0/2620 [00:00<?, ?ex/s]

In [19]:
# displaying the updated dataset
show_random_elements(dataset_train.remove_columns(["audio"]), num_examples=10)

,text
0,the brute has gone lame you see
1,your honour he ventured imploringly since your terror seems to have paralyzed your tongue said chauvelin sarcastically i must needs refresh your memory
2,yet if our hostess were to see us marching out with an umbrella how odd she would think us where are you off to she would ask and we could only answer lamely
3,at the close of a blue summer's day when the dreamy golden haze wrapped the city in a mystic charm i called at the studio having heard that he had returned and was settling down to work when i entered dick was standing before his easel
4,might be expected to lighten it on the contrary many a people has gradually emerged from this condition by the aid of a central authority whose position has made it the rival and has ended by making it the master of the local despots
5,he bowed to her oftener than to any of the rest and he thought she noticed it they danced until they were almost too tired to move
6,is dry for mead this is a bare and cold and hungry land i will visit my friends in iceland and our swords are rusty with long resting said his men perhaps we can find play for them in iceland
7,ankle deep in mud for the late rains had well moistened the black prairie soil it was a unique and picturesque procession
8,and so when bud's clothes were found at the swimming hole one day and no bud inside them she didn't take on up to the expectations of the neighbors who had brought the news and who were standing around waiting for her to go off into something special in the way of high strikes
9,i squeezed it in an involuntary convulsion all at once carried away by its frightful excess load the nautilus sank into the waters like a cannonball in other words dropping as if in a vacuum


In [20]:
# extracting the characters to create different vocabularies for training and testing
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [21]:
# different vocabularies
vocab_train = dataset_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dataset_train.column_names)
vocab_test = dataset_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dataset_test.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [22]:
# list of unique elements
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [23]:
# enumerating the vocabulary list using the key-value pair to get the vocab_dict
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{' ': 21,
 "'": 9,
 'a': 10,
 'b': 11,
 'c': 22,
 'd': 7,
 'e': 17,
 'f': 13,
 'g': 4,
 'h': 19,
 'i': 16,
 'j': 26,
 'k': 12,
 'l': 0,
 'm': 23,
 'n': 2,
 'o': 25,
 'p': 5,
 'q': 6,
 'r': 14,
 's': 27,
 't': 20,
 'u': 15,
 'v': 24,
 'w': 1,
 'x': 3,
 'y': 18,
 'z': 8}

In [24]:
# removing the delimiter
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [25]:
# Adding an unknown token and padding to generate the same length of the vocab_dict
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

30

In [26]:
# creating the json file from vocab_dict and vocab_file
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [27]:
# importing the wav2vec2CTCTokenizer to generate the tokenizer
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [28]:
# importing the wav2vec2FeatureExtractor to extract the features
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [29]:
# importing the wav2Vec2Processor to process the features and tokenizers
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [30]:
# Resampling of the audio file (Wav2vec = 1 input)
dataset_train[0]["audio"]

{'array': array([-0.00650024, -0.00552368, -0.00619507, ...,  0.0032959 ,
         0.00045776, -0.00946045], dtype=float32),
 'path': '/root/.cache/huggingface/datasets/downloads/extracted/39bad2659671b430f631158d09f2c89e593891e6d95eed4dbdda430301560fc0/LibriSpeech/train-clean-100/103/1240/103-1240-0000.flac',
 'sampling_rate': 16000}

In [31]:
# Resampling of the audio file
dataset_train = dataset_train.cast_column("audio", Audio(sampling_rate=16_000))
dataset_test = dataset_test.cast_column("audio", Audio(sampling_rate=16_000))

In [32]:
dataset_train[0]["audio"]

{'array': array([-0.00650024, -0.00552368, -0.00619507, ...,  0.0032959 ,
         0.00045776, -0.00946045], dtype=float32),
 'path': '/root/.cache/huggingface/datasets/downloads/extracted/39bad2659671b430f631158d09f2c89e593891e6d95eed4dbdda430301560fc0/LibriSpeech/train-clean-100/103/1240/103-1240-0000.flac',
 'sampling_rate': 16000}

In [33]:
# importing the ipd package to play the audio file 
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(dataset_train)-1)

ipd.Audio(data=dataset_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)

In [34]:
# preparing the target text and calculating the array shape and sampling rate
rand_int = random.randint(0, len(dataset_train)-1)

print("Target text:", dataset_train[rand_int]["text"])
print("Input array shape:", dataset_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", dataset_train[rand_int]["audio"]["sampling_rate"])

Target text: you might not be back in time in time for what oh how can you ask she rejoined in a sharp tone of reproach you know too well in time to make me your wife when the divorce shall appear i shall chance it 
Input array shape: (237760,)
Sampling rate: 16000


In [35]:
# preparing the dataset to pass the audio files in batches using the processor
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

In [36]:
# applying the map and all the necessary data preparation techniques 
dataset_train = dataset_train.map(prepare_dataset, remove_columns=dataset_train.column_names, num_proc=4)
dataset_test = dataset_test.map(prepare_dataset, remove_columns=dataset_test.column_names, num_proc=4)

In [37]:
# importing the packages and creating the datacollator having different features
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [38]:
#data collator
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [39]:
# importing the wer metric
wer_metric = load_metric("wer")

Downloading:   0%|          | 0.00/1.95k [00:00<?, ?B/s]

In [40]:
# defining a function to get the sequence of logit vectors
# Encoding the tokens with -100
# Decoding and ensure that the tokens are not grouped
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [41]:
# importing the pretrained model
# pad token id must be defined
# specify the parameters to save the gpu memory
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Downloading:   0%|          | 0.00/1.73k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['quantizer.weight_proj.bias', 'project_q.bias', 'project_q.weight', 'project_hid.bias', 'project_hid.weight', 'quantizer.codevectors', 'quantizer.weight_proj.weight']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to u

In [42]:
# freeze feature extractor
model.freeze_feature_extractor()

In [43]:
# gradient checkpoint enabled to save the memory and the resources
model.gradient_checkpointing_enable()

In [44]:
# Specifying the parameters for the training
from transformers import TrainingArguments

training_args = TrainingArguments(
  # output_dir="/content/gdrive/MyDrive/wav2vec2-large-xlsr-turkish-demo",
  output_dir="./wav2vec2-large-xlsr-turkish-demo",
  group_by_length=True,
  per_device_train_batch_size=4,
  gradient_accumulation_steps=16,
  evaluation_strategy="steps",
  num_train_epochs=2,
  save_steps=200,
  eval_steps=200,
  logging_steps=200,
  learning_rate=3e-4,
  warmup_steps=1000,
  save_total_limit=2,
)

In [73]:
# let us pass all the instances and start the training!
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset = dataset_train,
    eval_dataset=dataset_test,
    tokenizer=processor.feature_extractor,
)

```javascript
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}
setInterval(ConnectButton,60000);
```

In [46]:
#importing the packages
import torch
torch.cuda.empty_cache()

In [47]:
#importing the torch.nn containers (convolution)
import torch.nn as nn

In [48]:
#checking the availability of gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model= nn.DataParallel(model)
model.to(device)

DataParallel(
  (module): Wav2Vec2ForCTC(
    (wav2vec2): Wav2Vec2Model(
      (feature_extractor): Wav2Vec2FeatureExtractor(
        (conv_layers): ModuleList(
          (0): Wav2Vec2LayerNormConvLayer(
            (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
            (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          )
          (1): Wav2Vec2LayerNormConvLayer(
            (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
            (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          )
          (2): Wav2Vec2LayerNormConvLayer(
            (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
            (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          )
          (3): Wav2Vec2LayerNormConvLayer(
            (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
            (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          )
          (4): Wav2Vec2Layer

In [49]:
# training the model
trainer.train()

***** Running training *****
  Num examples = 31242
  Num Epochs = 2
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 16
  Total optimization steps = 976
/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:158: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  tensor = as_tensor(value)
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:882: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, round

Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss,Wer
200,5.336100,2.913172,0.999981
400,2.836000,2.403556,1.000000
600,0.775100,0.179095,0.199635
800,0.250600,0.115633,0.139664


***** Running Evaluation *****
  Num examples = 2620
  Batch size = 8
Saving model checkpoint to ./wav2vec2-large-xlsr-turkish-demo/checkpoint-200
Configuration saved in ./wav2vec2-large-xlsr-turkish-demo/checkpoint-200/config.json
Model weights saved in ./wav2vec2-large-xlsr-turkish-demo/checkpoint-200/pytorch_model.bin
Configuration saved in ./wav2vec2-large-xlsr-turkish-demo/checkpoint-200/preprocessor_config.json
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:882: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1
***** Running Evaluation *****
  Num examples = 2620
  Ba

TrainOutput(global_step=976, training_loss=1.9213601057646705, metrics={'train_runtime': 30039.015, 'train_samples_per_second': 2.08, 'train_steps_per_second': 0.032, 'total_flos': 2.3396389827763016e+19, 'train_loss': 1.9213601057646705, 'epoch': 2.0})

In [68]:
# evaluating the model
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)
  
  return batch

results = dataset_test.map(map_to_result, remove_columns=dataset_test.column_names)

  0%|          | 0/2620 [00:00<?, ?ex/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


In [69]:
# result
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))

Test WER: 0.195
